# Fine-tuning to predict the driver gene on example dataset

## Download pre-trained weights and example datasets

In [ ]:
import requests


url = "https://uc0c03bb31fdfb59f1d7ecd90ce9.dl.dropboxusercontent.com/cd/0/get/ClJWTv0BXBQWYcgjiMhP-0lhaUzC3Cv1V0ZjbNfHaKo4_XF0sEQKn6b50G2OIyY68SLa15qzBMyR11CE5_DUrWeXOnJOWB8xS_6r7oEiHSDdAtZkZpAaindvcQfW7qP-S4qg8EH2IwXWjwcYfoMSaQeiVmSAmidL-F9yFUEE9w9Zcw/file?_download_id=0038548811966447882196906360010739990789650147627526241001215657358&_log_download_success=1#"

save_path = "../data/"


response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(save_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
    print(f"Successfully downloaded to {save_path}")
else:
    print(f"Error: {response.status_code}")

## Parameter tuning

In [8]:
import json

params = {
    "global_batch_size": 128,
    "local_batch_size": 1,
    "mixed_precision": "true",
    "nr_step": 3000,
    "warmup_step": 500,
    "lr": 0.001,
    "chk_time_interval": 3600,
    "chk_step_interval": 100,
    
    "dataset_dir": "../data/",
    "log_dir": "../data/log",
    "model_dir": "../data/finetune/model",
    "pretrain_model_dir": "../data/pretrain",

    "train_data": "Re-stimulated_t_example_train.h5ad",
    "test_data": "Resting_t_example_test.h5ad",
    "dist_graph" : "dist_t_matrix.csv",
    "adj_graph" : "adj_t_matrix.csv"

}


with open('../config.json', 'w') as f:
    json.dump(params, f, indent=4)

## Training

### Check training dataset

In [4]:
import scanpy as sc

adata_train = sc.read_h5ad(params['dataset_dir']+params['train_data'])

## The cell by gene matrix should be raw counts.
adata_train

AnnData object with n_obs × n_vars = 5690 × 19240
    obs: 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'condition', 'guide_id', 'gene', 'gene_category', 'crispr', 'donor', 'percent.mt', 'percent.ribo', 'nCount_SCT', 'nFeature_SCT', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'CD4.CD8.Score', 'CD4.or.CD8', 'SCT_snn_res.0.4', 'seurat_clusters', 'cluster_name', 'activation.score', 'perturbation'
    var: 'gene_ids', 'n_cells'

In [5]:
## The perturbation label is stored in adata.obs.perturbation
adata_train.obs.perturbation

CGGAATTAGACTTCAC-6         IFNG
ATTACCTAGGAGATAG-5       INPPL1
TGGGTTAGTTGTATGC-5         GRAP
CTCCCTCTCGGTAGGA-8       P2RY14
GACCTTCTCATCTATC-6        IKZF3
                        ...    
TTCAGGACAGCAATTC-6      ARHGDIB
AGACAAACAAGAGCTG-5        IL2RB
AACCAACCAGACTGCC-7     APOBEC3C
CATGCAAGTACATACC-5         CD28
ACGGTTAGTACGATCT-8    NO-TARGET
Name: perturbation, Length: 5690, dtype: category
Categories (70, object): ['ABCB10', 'AKAP12', 'ALX4', 'APOBEC3C', ..., 'TRAF3IP2', 'TRIM21', 'VAV1', 'WT1']

### Run training scripts

In [7]:
## Single GPU training
%run start_train.py

CUDA is available. Number of GPUs: 1
dataset info: ../data/Re-stimulated_t_example_train.h5ad 5690
dataset info: ../data/Resting_t_example_test.h5ad 3158
Train Step: [1010/3000], ce_loss: 2.044270, update_loss: 2.044270, top1_acc: 0.502344, top5_acc: 0.723047, top10_acc: 0.845443, top50_acc: 0.990885, top100_acc: 1.000000, Speed: 0.051 m/s, Passed: 0.054 h, Estimate: 10.720 h


KeyboardInterrupt: 

## Testing 

### Load results and evaluate. 

In [4]:
## Here we load results on step 1000 as an example. 

%run load_results.py -c 1000

/mnt/isilon/tan_lab/pany3/CellNavi/tutorials/load_results.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.de

dataset info: /home/pany3/pany3/CellNavi/dataset_full/set3_example_test.h5ad 3158


100%|██████████| 3158/3158 [16:09<00:00,  3.26it/s]


In [5]:
import pandas as pd
import scanpy as sc
from sklearn.metrics import accuracy_score, f1_score
import numpy as np


df = pd.read_csv('set3_test_example_results.csv', index_col=0)

## The rows represent cell names and columns represent perturbed genes, with each value indicating the logits.
print(df.head())


                      ABCB10    AKAP12      ALX4  APOBEC3C  APOBEC3D  \
TGCATCCTCGATCCAA-4 -7.821463 -1.156066  0.904565 -5.127326 -7.951521   
TTGGATGGTATCCTCC-2 -7.477587 -1.947112 -1.975892 -1.696544 -7.003691   
AACCACAGTCTCCCTA-1 -9.641579 -3.978698 -5.091012 -5.724802 -5.395859   
TAGAGTCTCATGGATC-4 -1.639565 -2.339934  0.846601 -3.660203 -7.996193   
TTTCAGTTCCATTCGC-2 -3.726980 -3.369183 -4.184911  0.105774 -1.985862   

                       APOL2   ARHGDIB    BICDL2      CBY1       CD2  ...  \
TGCATCCTCGATCCAA-4 -1.957271  2.360333 -5.758391 -6.885813 -9.281010  ...   
TTGGATGGTATCCTCC-2 -2.546962  1.258157 -2.950146 -0.769823 -4.229147  ...   
AACCACAGTCTCCCTA-1 -5.997128 -5.460759 -0.255089 -5.033724 -3.466392  ...   
TAGAGTCTCATGGATC-4 -2.124491  1.050665 -0.941778  1.620585 -2.813236  ...   
TTTCAGTTCCATTCGC-2 -2.452409  2.454847 -1.646882 -0.831029 -1.429325  ...   

                       TAGAP     TBX21  TNFRSF1A   TNFRSF1B   TNFRSF9  \
TGCATCCTCGATCCAA-4 -9.457092 -5

In [6]:
adata_test = sc.read_h5ad(params['dataset_dir']+params['test_data'])
perturb_gene = adata_test.obs['perturbation'].values

df['pred_gene'] = df.idxmax(axis=1)
pred_gene = df['pred_gene'].values


accuracy = accuracy_score(perturb_gene, pred_gene)
f1 = f1_score(perturb_gene, pred_gene, average='weighted')


print(f"Testing accuracy (for toy dataset): {accuracy:.3f}")
print(f"Testing F1 score (for toy dataset): {f1:.3f}")

Testing accuracy (for toy dataset): 0.510
Testing F1 score (for toy dataset): 0.498
